In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from auction.operations import *
from store.operations import StoringPool
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')
print('STORE_APP_ID', os.environ.get('STORE_APP_ID'))

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

STORE_APP_ID 3310


In [3]:
staking_app = Account.from_mnemonic(os.environ.get("CREATOR_MN"))
team_wallet = Account.from_mnemonic(os.environ.get("TEAM_MN"))

creator = Account.from_mnemonic(os.environ.get("CREATOR_MN"))
seller = Account.from_mnemonic(os.environ.get("SELLER_MN"))

bidder = Account.from_mnemonic(os.environ.get("BUYER_MN"))
bidder1 = Account.from_mnemonic(os.environ.get("BUYER1_MN"))
bidder2 = Account.from_mnemonic(os.environ.get("BUYER2_MN"))

print(f"Staking App Address: {staking_app.get_address()}")
print(f"Team wallet Address: {team_wallet.get_address()}")
print(f"Creator address: {creator.get_address()}")
print(f"Seller Address: {seller.get_address()}")
print(f"Bidder Address: {bidder.get_address()}")
print(f"Bidder1 Address: {bidder1.get_address()}")
print(f"Bidder2 Address: {bidder2.get_address()}")   
  


Staking App Address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Team wallet Address: UCAX3M5HS7PWWLB6HDCM2O7JE4HC3RGBPNWFRP4SYU62ZS5NSS2GFOKIOU
Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Seller Address: CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA
Bidder Address: X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Bidder1 Address: LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I
Bidder2 Address: 742N2DLQ6NUUQXNUTNIJHYXYGL6BWG7C2XSQDHLFJN2NFYDVAYYUAQ36NQ


In [ ]:
#seller_info = get_account_info(client, seller.get_address())
#print('seller_info', seller_info)

In [ ]:
deleteApps(client, [], creator)
# optoutApps(client, [], seller)
# optoutApps(client, [], bidder)
# optoutApps(client, [], bidder1)
# optoutApps(client, [], bidder2)

Alice is generating an example token...

In [4]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

# asset_id = create_dummy_asset(creator, 1000000000000000000, 3, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")
#send_asset(client, asset_id, 500000000000000, creator, seller)

Waiting for confirmation...
Transaction WX63JYWDD5Z6MTFYL2WF6PXMWWVXYDKEOF2Y3RJVC3TSOVVFAFGQ confirmed in round 136234.
The token id is: 2989


In [ ]:
start_time = int(time()) + 10
end_time = start_time + 30
print(f"start_time", start_time)
reserve = 1_000_000  # 1 Algo
token_amount = 100
increment = 100_000  # 0.1 Algo

In [ ]:
storeApp = StoringPool(client=client, creator=creator)

storeAppID = int(os.environ.get("STORE_APP_ID"))
storeApp.app_id = storeAppID

# optout_app(client, storeAppID, seller)
# optout_app(client, storeAppID, bidder)
# optout_app(client, storeAppID, bidder1)
# optout_app(client, storeAppID, bidder2)

storeApp.create_app()
storeAppID = storeApp.app_id

print(f"storeAppID", storeAppID)
print(b"storeApp address", get_application_address(storeAppID))


"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [ ]:

print(f"seller address: {seller.get_address()}")
appID = create_auction_app(
    client=client,
    creator=creator,
    staking_address=staking_app.get_address(),
    team_wallet_address=team_wallet.get_address(),
    store_app_id=storeAppID
)

#appID = 2843
print(f"Auction App ID: {appID}")
print(f"Auction App Address: {get_application_address(appID)}")
print("Latest time:", get_app_global_state(client, appID))

Alice is setting up and funding token auction...

In [ ]:
storeApp.set_up(trade_app_id=100, bid_app_id=101, auction_app_id=appID)

In [ ]:
auction_index = setup_auction_app(
    client=client,
    app_id=appID,
    seller=seller,
    token_id=asset_id,
    token_amount=token_amount,
    start_time=start_time,
    end_time=end_time,
    reserve=reserve,
    min_bid_increment=increment
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

In [ ]:
close_auction(client, appID, auction_index, seller)

In [ ]:
start_time = int(time()) + 10
end_time = start_time + 30
print(f"start_time", start_time)
auction_index = setup_auction_app(
    client=client,
    app_id=appID,
    seller=seller,
    token_id=asset_id,
    token_amount=token_amount,
    start_time=start_time,
    end_time=end_time,
    reserve=reserve,
    min_bid_increment=increment
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

In [ ]:
# _, last_round_time = get_last_block_timestamp(client)
# print(f"start_time", start_time)
# print(f"last_round_time", last_round_time)
# if last_round_time < start_time + 5:
#     sleep(start_time + 5 - last_round_time)

actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

In [ ]:
bidAmount = reserve + 4000
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidAmount, " algos")

In [ ]:
_, lastRoundTime = get_last_block_timestamp(client)
print(f"lastRoundTime", lastRoundTime)
place_bid(client=client, app_id=appID, auction_index=auction_index, bidder=bidder, bid_amount=bidAmount)

print("Carla is opting into token with id:", asset_id)

In [ ]:
bidAmount = reserve + 300_000
place_bid(client=client, app_id=appID, auction_index=auction_index, bidder=bidder1, bid_amount=bidAmount)

bidAmount = reserve + 500_000
place_bid(client=client, app_id=appID, auction_index=auction_index, bidder=bidder2, bid_amount=bidAmount)


Alice is closing out the auction....

In [ ]:
# _, lastRoundTime = get_last_block_timestamp(client)
# if lastRoundTime < end_time + 5:
#     sleep(end_time + 5 - lastRoundTime)

close_auction(client, appID, auction_index, seller)

In [ ]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)
print(f"storeAppID", storeAppID)
#assert actualAppBalances == expectedAppBalances

from utils import get_app_local_state
print(get_app_local_state(client, storeAppID, seller.get_address()))
print(get_app_local_state(client, storeAppID, bidder.get_address()))
print(get_app_local_state(client, storeAppID, bidder1.get_address()))
print(get_app_local_state(client, storeAppID, bidder2.get_address()))

In [ ]:
#bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]
bidder1NftBalance = get_balances(client, bidder1.get_address())[asset_id]
#bidder2NftBalance = get_balances(client, bidder2.get_address())[asset_id]

print("Carla's NFT balance:", bidder1NftBalance, " for token ID: ", asset_id)

assert bidder1NftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")
actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidAmount - 1_000
assert actualSellerBalances[asset_id] == 0

In [ ]:
storeAppState = get_app_global_state(client, storeAppID)
print(storeAppState)
print(encoding.encode_address(storeAppState[b"C"]))